# 0. Load Files

In [1]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import PCA

import joblib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_parquet('C://Users/Isi/anaconda3/envs/FireBrigade/MAY24_BDS_INT_Fire_Brigade/data/df_cleaned_for_classification_models.parquet')
df.head() 

,IncidentGroup_Fire,IncidentGroup_Special Service,AggregatedPropertyCategory_Outdoor,AggregatedPropertyCategory_Residential,AggregatedPropertyCategory_Vehicle,CellEastingNorthing2500_502500-175000,CellEastingNorthing2500_502500-177500,CellEastingNorthing2500_502500-180000,CellEastingNorthing2500_502500-182500,CellEastingNorthing2500_502500-187500,CellEastingNorthing2500_502500-190000,CellEastingNorthing2500_502500-192500,CellEastingNorthing2500_505000-172500,CellEastingNorthing2500_505000-175000,CellEastingNorthing2500_505000-177500,CellEastingNorthing2500_505000-180000,CellEastingNorthing2500_505000-182500,CellEastingNorthing2500_505000-185000,CellEastingNorthing2500_505000-187500,CellEastingNorthing2500_505000-190000,CellEastingNorthing2500_507500-170000,CellEastingNorthing2500_507500-172500,CellEastingNorthing2500_507500-175000,CellEastingNorthing2500_507500-177500,CellEastingNorthing2500_507500-180000,CellEastingNorthing2500_507500-182500,CellEastingNorthing2500_507500-185000,CellEastingNorthing2500_507500-187500,CellEastingNorthing2500_507500-190000,CellEastingNorthing2500_510000-167500,CellEastingNorthing2500_510000-170000,CellEastingNorthing2500_510000-172500,CellEastingNorthing2500_510000-175000,CellEastingNorthing2500_510000-177500,CellEastingNorthing2500_510000-180000,CellEastingNorthing2500_510000-182500,CellEastingNorthing2500_510000-185000,CellEastingNorthing2500_510000-187500,CellEastingNorthing2500_510000-190000,CellEastingNorthing2500_512500-167500,CellEastingNorthing2500_512500-170000,CellEastingNorthing2500_512500-172500,CellEastingNorthing2500_512500-175000,CellEastingNorthing2500_512500-177500,CellEastingNorthing2500_512500-180000,CellEastingNorthing2500_512500-182500,CellEastingNorthing2500_512500-185000,CellEastingNorthing2500_512500-187500,CellEastingNorthing2500_512500-190000,CellEastingNorthing2500_512500-192500,CellEastingNorthing2500_515000-157500,CellEastingNorthing2500_515000-160000,CellEastingNorthing2500_515000-162500,CellEastingNorthing2500_515000-165000,CellEastingNorthing2500_515000-167500,CellEastingNorthing2500_515000-170000,CellEastingNorthing2500_515000-172500,CellEastingNorthing2500_515000-175000,CellEastingNorthing2500_515000-177500,CellEastingNorthing2500_515000-180000,CellEastingNorthing2500_515000-182500,CellEastingNorthing2500_515000-185000,CellEastingNorthing2500_515000-187500,CellEastingNorthing2500_515000-190000,CellEastingNorthing2500_515000-192500,CellEastingNorthing2500_517500-160000,CellEastingNorthing2500_517500-162500,CellEastingNorthing2500_517500-165000,CellEastingNorthing2500_517500-167500,CellEastingNorthing2500_517500-170000,CellEastingNorthing2500_517500-172500,CellEastingNorthing2500_517500-175000,CellEastingNorthing2500_517500-177500,CellEastingNorthing2500_517500-180000,CellEastingNorthing2500_517500-182500,CellEastingNorthing2500_517500-185000,CellEastingNorthing2500_517500-187500,CellEastingNorthing2500_517500-190000,CellEastingNorthing2500_517500-192500,CellEastingNorthing2500_517500-195000,CellEastingNorthing2500_520000-162500,CellEastingNorthing2500_520000-165000,CellEastingNorthing2500_520000-167500,CellEastingNorthing2500_520000-170000,CellEastingNorthing2500_520000-172500,CellEastingNorthing2500_520000-175000,CellEastingNorthing2500_520000-177500,CellEastingNorthing2500_520000-180000,CellEastingNorthing2500_520000-182500,CellEastingNorthing2500_520000-185000,CellEastingNorthing2500_520000-187500,CellEastingNorthing2500_520000-190000,CellEastingNorthing2500_520000-192500,CellEastingNorthing2500_520000-195000,CellEastingNorthing2500_522500-160000,CellEastingNorthing2500_522500-162500,CellEastingNorthing2500_522500-165000,CellEastingNorthing2500_522500-167500,CellEastingNorthing2500_522500-170000,CellEastingNorthing2500_522500-172500,CellEastingNorthing2500_522500-175000,CellEastingNorthing2500_522500-177500,CellEastingNorthing2500_522500-180000,CellEastingNorthing2500_522500-182500,CellEastingNorthing2500_522500-185000,CellEastingNorthing2500_522500-187500,Cel

# 1. PCA Transformation & Balancing Data Set

In [3]:
X = df.drop("ResponseTimeBinary", axis = 1)
y = df["ResponseTimeBinary"]


In [4]:
pca = PCA(n_components=0.85)
X_pca = pca.fit_transform(X)

In [5]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size = 0.2, random_state = 666)


In [6]:
rUs = RandomUnderSampler(random_state=666)
X_ru, y_ru = rUs.fit_resample(X_train_pca, y_train_pca)
print('Sample classes undersampled :', dict(pd.Series(y_ru).value_counts()))


Sample classes undersampled : {0: 361454, 1: 361454}


# 2. Voting Classifier

## 2.1 Instantiating XGBoost, RF, & LogReg Models

In [7]:
xgboost = xgb.XGBClassifier(random_state=666)
rf = RandomForestClassifier(random_state=666)
logreg = LogisticRegression(random_state=666)

## 2.3 Cross Validation via  GridSearchCV

In [9]:
params_xgboost = {
    'XGboost__learning_rate': [0.1],
    'XGboost__max_depth': [5],
    'XGboost__n_estimators': [200],
    'XGboost__subsample': [0.9]
}
params_rf = {
    'RF__criterion': ['gini'],
    'RF__max_depth': [10],
    'RF__n_estimators': [200]
}
params_logreg = {
    'LogReg__C': [5],
    'LogReg__penalty': ['l2'],
    'LogReg__solver': ['lbfgs']
}

# Combine parameters into a single dictionary
params = {**params_xgboost, **params_rf, **params_logreg}

## 2.4 Model Evaluation (Voting Classifier (Soft))

In [24]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, balanced_accuracy_score

def evaluate_model(y_true, y_pred, y_proba = None):
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print('Confusion Matrix:\n', cm)
    
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy:.4f}')
    
    # Precision, Recall, F1-Score
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    
    print('Classification Report:\n', classification_report(y_true, y_pred, target_names=['Not Reached (>6 min)', 'Reached (<=6 min)']))
    
    # ROC AUC Score
    if y_proba is not None:
        roc_auc = roc_auc_score(y_true, y_proba[:, 1])
        print(f'ROC AUC Score: {roc_auc:.4f}')
    else:
        roc_auc = None
    
    # Balanced Accuracy
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    print(f'Balanced Accuracy: {balanced_acc:.4f}')
    
    return accuracy, precision, recall, f1, roc_auc, balanced_acc

## 2.5 Instantiating Voting Classifier (Hard)

In [14]:
vclf_hard = VotingClassifier(estimators=[('XGboost', xgboost), ('RF', rf), ('LogReg', logreg)], voting='hard')

## 2.6 Cross Validation via  GridSearchCV

In [15]:
vclf_hard_grid = GridSearchCV(estimator=vclf_hard, param_grid=params, cv=5)
vclf_hard_grid.fit(X_ru, y_ru)

GridSearchCV(cv=5,
             estimator=VotingClassifier(estimators=[('XGboost',
                                                     XGBClassifier(base_score=None,
                                                                   booster=None,
                                                                   callbacks=None,
                                                                   colsample_bylevel=None,
                                                                   colsample_bynode=None,
                                                                   colsample_bytree=None,
                                                                   device=None,
                                                                   early_stopping_rounds=None,
                                                                   enable_categorical=False,
                                                                   eval_metric=None,
                                                                   feature_types=None,
                                                                   gamma=None,
                                                                   grow_policy=None,
                                                                   importance_type=None,
                                                                   interact...
                                                     RandomForestClassifier(random_state=666)),
                                                    ('LogReg',
                                                     LogisticRegression(random_state=666))]),
             param_grid={'LogReg__C': [5], 'LogReg__penalty': ['l2'],
                         'LogReg__solver': ['lbfgs'], 'RF__criterion': ['gini'],
                         'RF__max_depth': [10], 'RF__n_estimators': [200],
                         'XGboost__learning_rate': [0.1],
                         'XGboost__max_depth': [5],
                         'XGboost__n_estimators': [200],
                         'XGboost__subsample': [0.9]})

In [20]:
# Save the best model
joblib.dump(vclf_hard_grid.best_estimator_, 'vclf_hard_grid.pkl')

# To load the model later
#vclf_hard_grid = joblib.load('C://Users/Isi/anaconda3/envs/FireBrigade/Pickles/vclf_hard_grid.pkl')

['vclf_hard_grid.pkl']

In [21]:
best_voting_clf_hard = vclf_hard_grid.best_estimator_

print("Best Voting Classifier model:")
print(best_voting_clf_hard)

Best Voting Classifier model:
VotingClassifier(estimators=[('XGboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learnin...
                                          

In [22]:
y_pred_train_vclf_hard = best_voting_clf_hard.predict(X_ru)
y_pred_test_vclf_hard = best_voting_clf_hard.predict(X_test_pca)


## 2.7 Model Evaluation (Voting Classifier (Hard))

In [25]:
print("Voting Classifier (Hard) - Training Data")
evaluate_model(y_ru, y_pred_train_vclf_hard)

Voting Classifier (Hard) - Training Data
Confusion Matrix:
 [[231778 129676]
 [101296 260158]]
Accuracy: 0.6805
Classification Report:
                       precision    recall  f1-score   support

Not Reached (>6 min)       0.70      0.64      0.67    361454
   Reached (<=6 min)       0.67      0.72      0.69    361454

            accuracy                           0.68    722908
           macro avg       0.68      0.68      0.68    722908
        weighted avg       0.68      0.68      0.68    722908

Balanced Accuracy: 0.6805


(0.6804959967243411,
 0.6816156199770143,
 0.6804959967243411,
 0.6800028173842438,
 None,
 0.6804959967243411)

In [25]:
print("Voting Classifier (Hard) - Training Data")
evaluate_model(y_ru, y_pred_train_vclf_hard)

Voting Classifier (Hard) - Training Data
Confusion Matrix:
 [[231778 129676]
 [101296 260158]]
Accuracy: 0.6805
Classification Report:
                       precision    recall  f1-score   support

Not Reached (>6 min)       0.70      0.64      0.67    361454
   Reached (<=6 min)       0.67      0.72      0.69    361454

            accuracy                           0.68    722908
           macro avg       0.68      0.68      0.68    722908
        weighted avg       0.68      0.68      0.68    722908

Balanced Accuracy: 0.6805


(0.6804959967243411,
 0.6816156199770143,
 0.6804959967243411,
 0.6800028173842438,
 None,
 0.6804959967243411)

In [27]:
print("\nVoting Classifier (Hard)  - Test Data")
evaluate_model(y_test_pca, y_pred_test_vclf_hard)


Voting Classifier (Hard)  - Test Data
Confusion Matrix:
 [[ 57384  32859]
 [ 61878 155420]]
Accuracy: 0.6920
Classification Report:
                       precision    recall  f1-score   support

Not Reached (>6 min)       0.48      0.64      0.55     90243
   Reached (<=6 min)       0.83      0.72      0.77    217298

            accuracy                           0.69    307541
           macro avg       0.65      0.68      0.66    307541
        weighted avg       0.72      0.69      0.70    307541

Balanced Accuracy: 0.6756


(0.6919532680195486,
 0.6533181065902882,
 0.6755610480791867,
 0.6571099053437849,
 None,
 0.6755610480791867)

#### Best Model:
-> Random Forest (Balanced + PCA + Hyperparameter Tuning)

### Best Parameter for RF

In [140]:
print(f'Best Parameters: {grid_search_rf_hyp_ru_pca.best_params_}')

Best Parameters: {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 200, 'random_state': 666}


In [144]:
best_model = grid_search_rf_hyp_ru_pca.best_estimator_
y_p = best_model.predict(X_test_pca)
balanced_acc = balanced_accuracy_score(y_test_pca, y_p)
print(f'Balanced Accuracy on Test Data: {balanced_acc:.4f}')

Balanced Accuracy on Test Data: 0.6733
